In [1]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Statistical analysis
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Machine learning (supervised learning)
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols

# Plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_1 = pd.read_excel("2021.xlsx")
df_2 = pd.read_excel("2022.xlsx")
df_concatenated = pd.concat([df_1, df_2], ignore_index=True)
# Display the first few rows of the concatenated DataFrame
print(df_concatenated.head())
# Display the shape of the concatenated DataFrame
print(f"Shape of concatenated DataFrame: {df_concatenated.shape}")
# Display the columns of the concatenated DataFrame
print(f"Columns in concatenated DataFrame: {df_concatenated.columns.tolist()}")
# Display the data types of the columns in the concatenated DataFrame
print(f"Data types of columns in concatenated DataFrame:\n{df_concatenated.dtypes}")
# Display summary statistics of the concatenated DataFrame
print(f"Summary statistics of concatenated DataFrame:\n{df_concatenated.describe()}")
# Check for missing values in the concatenated DataFrame
print(f"Missing values in concatenated DataFrame:\n{df_concatenated.isnull().sum()}")

      销售月份 一级平台  二级平台          店铺名称 品牌名称                 商品编码       销售金额  \
0  2021-04   抖音  抖音店铺       醉卿酒类旗舰店   其他  3466169139880691489   860440.0   
1  2021-04   抖音  抖音店铺      酒仙网卖场旗舰店  五粮液  3472827127450227804  1968300.0   
2  2021-04   抖音  抖音店铺      酒仙网卖场旗舰店  五粮液  3472877103781930055  1259252.0   
3  2021-04   抖音  抖音店铺  美狄丝华夏大地酒类专卖店   茅台  3449657277378912485   105316.0   
4  2021-04   抖音  抖音店铺      酒仙网卖场旗舰店  五粮液  3468580854253631057   779602.0   

    销售数量  
0  83879  
1  78732  
2  63279  
3  54538  
4  38245  
Shape of concatenated DataFrame: (1192252, 8)
Columns in concatenated DataFrame: ['销售月份', '一级平台', '二级平台', '店铺名称', '品牌名称', '商品编码', '销售金额', '销售数量']
Data types of columns in concatenated DataFrame:
销售月份     object
一级平台     object
二级平台     object
店铺名称     object
品牌名称     object
商品编码      int64
销售金额    float64
销售数量      int64
dtype: object
Summary statistics of concatenated DataFrame:
               商品编码          销售金额          销售数量
count  1.192252e+06  1.192252e+06  1.192252e+0

KeyError: 'month'